In [2]:
import pandas as pd
import sqlite3

In [3]:
data1 = pd.read_csv("latitude_longitude_details.csv")
data2 = pd.read_csv("terrain_classification_test.csv")

In [4]:
newdata2 = data2.append([data2]*118,ignore_index=True)

In [5]:
concatenated = pd.concat([data1, newdata2], axis=1)

In [6]:
concatenated.columns = ['latitude', 'longitude', 'terrain', 'distance']

In [7]:
conn = sqlite3.connect('test.db')

conn.execute('''CREATE TABLE TEST
         (latitude      INT      NOT NULL,
         longitude      INT    NOT NULL,
         terrain        TEXT     NOT NULL,
         distance       INT);''')

In [8]:
insert_statement = """
INSERT INTO TEST (latitude,
                    longitude,
                    terrain,
                    distance)
VALUES (?, ?, ?, ?)"""

In [11]:
conn = sqlite3.connect('test.db')
c = conn.cursor()

In [12]:
for i in range(len(concatenated)):
    values = tuple(concatenated.iloc[i])
    c.execute(insert_statement, values)

In [13]:
print (pd.read_sql_query("SELECT latitude,longitude,terrain FROM TEST WHERE ',' || terrain || ',' LIKE '%,' || 'road' || ',%'  OR ',' || terrain || ',' LIKE '%,' || 'boundary wall' || ',% ' AND ',' || terrain || ',' NOT LIKE '%,' || 'civil station' || ',%';", conn))

      latitude  longitude             terrain
0    10.006992  76.373570  boundary wall,road
1    10.006992  76.373608                road
2    10.007051  76.373335  boundary wall,road
3    10.007069  76.373379                road
4    10.007045  76.373344  boundary wall,road
..         ...        ...                 ...
233  10.017974  76.349704                road
234  10.017985  76.349716  boundary wall,road
235  10.017987  76.349719                road
236  10.017960  76.349727  boundary wall,road
237  10.017959  76.349723                road

[238 rows x 3 columns]


In [46]:
newdf = pd.read_sql_query("SELECT latitude,longitude,terrain FROM TEST WHERE ',' || terrain || ',' LIKE '%,' || 'road' || ',%'  OR ',' || terrain || ',' LIKE '%,' || 'boundary wall' || ',% 'OR ',' || terrain || ',' LIKE '%,' || 'river side' || ',%' AND ',' || terrain || ',' NOT LIKE '%,' || 'civil station' || ',%';", conn)

In [47]:
newdf.to_csv('query_result.csv',index=False)
